In [1]:
from openai import OpenAI

client = OpenAI(api_key="0", base_url="http://192.168.106.26:20000/v1")

def llm_qwen(prompt, model="Qwen3", temperature=0.7, top_p=0.8, max_tokens=2048, presence_penalty=1.5):
    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        presence_penalty=presence_penalty,
    )
    return chat_response.choices[0].message.content

if __name__ == "__main__":
    llm_output = llm_qwen("你好")
    print(llm_output)

你好！有什么我可以帮你的吗？😊


In [ ]:
import json
import nltk
import numpy as np
import torch
import concurrent.futures
import re
from typing import List, Tuple, Dict, Any, Optional, Union
from transformers import AutoModel, AutoTokenizer
from nltk.tokenize import word_tokenize
from collections import Counter

# 下载NLTK所需资源（首次运行时取消注释）
# nltk.download('punkt')
# nltk.download('punkt_tab')

class ComplexityWeightedAggregationLayer(torch.nn.Module):
    def __init__(self, hidden_size=768, num_perspectives=3, complexity_weight_ratio=0.7):
        """
        基于复杂度加权的聚合层：
        - 保留全局聚合路径
        - 用文本复杂度替代注意力权重
        - complexity_weight_ratio控制复杂度权重影响程度
        """
        super().__init__()
        self.num_perspectives = num_perspectives
        self.complexity_weight_ratio = complexity_weight_ratio
        
        # 第一级：两两视角聚合
        self.pairwise_aggregators = torch.nn.ModuleList([
            torch.nn.Sequential(
                torch.nn.Linear(hidden_size * 2, hidden_size),
                torch.nn.LayerNorm(hidden_size),
                torch.nn.GELU()
            )
            for _ in range(num_perspectives * (num_perspectives - 1) // 2)
        ])
        
        # 第二级：全局聚合
        num_pairwise = num_perspectives * (num_perspectives - 1) // 2
        self.global_aggregator = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_pairwise, hidden_size * 2),
            torch.nn.LayerNorm(hidden_size * 2),
            torch.nn.GELU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(hidden_size * 2, hidden_size)
        )
        
        # 文本复杂度计算模型（外部传入）
        self.complexity_model = None
        self.complexity_tokenizer = None
    
    def set_complexity_tools(self, model: AutoModel, tokenizer: AutoTokenizer):
        """设置复杂度计算所需的模型和分词器"""
        self.complexity_model = model
        self.complexity_tokenizer = tokenizer
    
    def forward(self, encoded_responses: List[torch.Tensor], response_texts: List[str]):
        """
        前向传播过程
        
        参数：
            encoded_responses: 视角特征列表 [num_perspectives, hidden_size]
            response_texts: 视角文本列表 [num_perspectives]
        """
        stacked = torch.stack(encoded_responses)  # [num_perspectives, hidden_size]
        
        # 第一阶段：两两视角聚合
        pairwise_outputs = []
        idx = 0
        for i in range(self.num_perspectives):
            for j in range(i + 1, self.num_perspectives):
                pair = torch.cat([stacked[i], stacked[j]], dim=-1).unsqueeze(0)
                pairwise_outputs.append(self.pairwise_aggregators[idx](pair))
                idx += 1
        
        # 第二阶段：全局聚合
        pairwise_concat = torch.cat(pairwise_outputs, dim=-1)  # [1, hidden_size*num_pairs]
        global_repr = self.global_aggregator(pairwise_concat)  # [1, hidden_size]
        
        # 第三阶段：基于复杂度的权重计算
        complexities = [
            calculate_complexity(text, self.complexity_model, self.complexity_tokenizer)
            for text in response_texts
        ]
        
        # 归一化为概率分布
        complexities = torch.tensor(complexities, dtype=torch.float32)
        weights = torch.softmax(complexities, dim=-1)  # [num_perspectives]
        
        # 原始视角的加权和
        weighted_aggregation = torch.sum(
            stacked * weights.t().unsqueeze(-1),
            dim=0
        )  # [hidden_size]
        
        # 最终输出融合
        final_output = (self.complexity_weight_ratio * weighted_aggregation.unsqueeze(0) + 
                       (1 - self.complexity_weight_ratio) * global_repr)  # [1, hidden_size]
        
        return final_output, weights

def load_encoder_model() -> Tuple[AutoModel, AutoTokenizer]:
    """加载预训练的BERT编码器和分词器"""
    model_name = "/app/sda1/xiangyue/model/bert-base-chinese"
    return AutoModel.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name)

def encode_text(text: str, model: AutoModel, tokenizer: AutoTokenizer) -> torch.Tensor:
    """将文本编码为BERT的CLS向量"""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        return model(**inputs).last_hidden_state[:, 0, :].squeeze(0)  # [hidden_size]

def calculate_complexity(text: str, model: AutoModel, tokenizer: AutoTokenizer) -> float:
    """
    计算文本复杂度（多维度指标融合）：
    1. 文本长度复杂度
    2. 词汇多样性（TF-IDF熵）
    3. 语义嵌入方差
    """
    if not text:
        return 0.0
    
    # 1. 文本长度复杂度（对数归一化）
    word_count = len(word_tokenize(text))
    length_score = np.log(word_count + 1) / np.log(100 + 1)  # 归一化到[0,1]
    
    # 2. 词汇多样性（TF-IDF熵）
    words = [word for word in word_tokenize(text) if word.isalpha() and len(word) > 1]
    if len(words) < 2:
        diversity_score = 0.0
    else:
        word_freq = Counter(words)
        probs = np.array(list(word_freq.values()), dtype=float) / len(words)
        diversity_score = -np.sum(probs * np.log(probs + 1e-10)) / np.log(len(word_freq))
        diversity_score = max(0.0, min(1.0, diversity_score))  # 裁剪到[0,1]
    
    # 3. 语义嵌入方差（BERT各token向量的方差）
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.squeeze(0)
        valid_mask = inputs.input_ids.squeeze(0) != 0  # 忽略padding
        valid_embeddings = embeddings[valid_mask]
        if len(valid_embeddings) < 2:
            semantic_variance = 0.0
        else:
            variance = torch.var(valid_embeddings, dim=0).mean().item()
            # 动态范围归一化（假设方差通常在0.01-1.0之间）
            semantic_variance = (variance - 0.01) / (1.0 - 0.01) if variance > 0.01 else 0.0
            semantic_variance = max(0.0, min(1.0, semantic_variance))  # 裁剪到[0,1]
    
    # 综合复杂度分数（各指标加权求和）
    complexity_score = 0.3 * length_score + 0.3 * diversity_score + 0.4 * semantic_variance
    return max(complexity_score, 1e-6)  # 防止全零

def process_perspective(
    index: int, 
    question: str, 
    encoder_model: AutoModel, 
    tokenizer: AutoTokenizer, 
    llm: callable, 
    temperature: float
) -> Tuple[int, str, torch.Tensor]:
    """处理单个视角：生成回答 -> 编码向量"""
    response = llm(f"{question}", temperature=temperature)
    encoded = encode_text(response, encoder_model, tokenizer)
    return index, response, encoded

def multi_perspective_analysis(
    metaprompt: str, 
    p: int = 3, 
    topk: int = 1, 
    llm: callable = None, 
    temperature_settings: List[float] = None,
    complexity_weight_ratio: float = 0.4,
    hidden_size: int = 768,
    length_weight: float = 0.3,
    diversity_weight: float = 0.3,
    variance_weight: float = 0.4,
) -> Dict[str, Any]:
    """
    多视角分析主函数：
    1. 并行生成p个视角回答
    2. 计算各视角复杂度权重
    3. 聚合特征并返回topk视角
    
    参数:
        metaprompt: 元提示文本
        p: 视角数量
        topk: 返回的顶部视角数量
        llm: 大语言模型调用函数
        temperature_settings: 各视角的温度设置
        complexity_weight_ratio: 复杂度权重占比
        hidden_size: 隐藏层大小
        length_weight: 文本长度复杂度权重
        diversity_weight: 词汇多样性权重
        variance_weight: 语义嵌入方差权重
    """
    global calculate_complexity
    
    if llm is None:
        llm = llm_qwen  # 默认使用模拟LLM
    
    # 处理temperature设置
    if temperature_settings is None:
        temperatures = [0.1] * p
    elif isinstance(temperature_settings, (int, float)):
        temperatures = [temperature_settings] * p
    elif len(temperature_settings) == p:
        temperatures = temperature_settings
    else:
        raise ValueError("temperature_settings需为单个值或长度为p的列表")
    
    # 初始化模型
    encoder_model, tokenizer = load_encoder_model()
    
    # 创建闭包以修改复杂度计算的权重
    original_calculate_complexity = calculate_complexity
    def calculate_complexity_with_weights(text: str, model: AutoModel, tokenizer: AutoTokenizer) -> float:
        """使用自定义权重的文本复杂度计算函数"""
        if not text:
            return 0.0
        
        # 1. 文本长度复杂度
        word_count = len(word_tokenize(text))
        length_score = np.log(word_count + 1) / np.log(100 + 1)
        
        # 2. 词汇多样性
        words = [word for word in word_tokenize(text) if word.isalpha() and len(word) > 1]
        if len(words) < 2:
            diversity_score = 0.0
        else:
            word_freq = Counter(words)
            probs = np.array(list(word_freq.values()), dtype=float) / len(words)
            diversity_score = -np.sum(probs * np.log(probs + 1e-10)) / np.log(len(word_freq))
            diversity_score = max(0.0, min(1.0, diversity_score))
        
        # 3. 语义嵌入方差
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            embeddings = model(**inputs).last_hidden_state.squeeze(0)
            valid_mask = inputs.input_ids.squeeze(0) != 0
            valid_embeddings = embeddings[valid_mask]
            if len(valid_embeddings) < 2:
                semantic_variance = 0.0
            else:
                variance = torch.var(valid_embeddings, dim=0).mean().item()
                semantic_variance = (variance - 0.01) / (1.0 - 0.01) if variance > 0.01 else 0.0
                semantic_variance = max(0.0, min(1.0, semantic_variance))
        
        # 使用传入的权重
        complexity_score = (length_weight * length_score + 
                           diversity_weight * diversity_score + 
                           variance_weight * semantic_variance)
        return max(complexity_score, 1e-6)
    
    # 临时替换复杂度计算函数
    calculate_complexity = calculate_complexity_with_weights
    
    aggregation_model = ComplexityWeightedAggregationLayer(
        num_perspectives=p, 
        complexity_weight_ratio=complexity_weight_ratio,
        hidden_size=hidden_size
    )
    aggregation_model.set_complexity_tools(encoder_model, tokenizer)
    aggregation_model.eval()  # 设置为评估模式
    
    # 并行处理多视角
    with concurrent.futures.ThreadPoolExecutor(max_workers=p) as executor:
        futures = [
            executor.submit(
                process_perspective, 
                i, 
                metaprompt, 
                encoder_model, 
                tokenizer, 
                llm, 
                temperatures[i]
            ) for i in range(p)
        ]
        results = [future.result() for future in futures]
    
    # 恢复原始的复杂度计算函数
    calculate_complexity = original_calculate_complexity
    
    # 整理结果
    results.sort(key=lambda x: x[0])
    encoded_responses = [enc for _, _, enc in results]
    response_texts = [resp for _, resp, _ in results]
    
    # 打印各视角信息（含复杂度）
    print("\n=== 各视角分析结果 ===")
    complexities = [
        calculate_complexity_with_weights(text, encoder_model, tokenizer)
        for text in response_texts
    ]
    for idx, (text, temp, comp) in enumerate(zip(response_texts, temperatures, complexities), start=1):
        # print(f"视角{idx} (temp={temp:.2f}, 复杂度={comp:.4f}):")
        print(f"视角{idx} temp={temp:.2f}")
        print(f"  {text}\n")
    
    if not encoded_responses:
        return {"error": "未获取到有效视角"}
    
    # 执行复杂度加权聚合
    with torch.no_grad():
        aggregated_output, weights = aggregation_model(encoded_responses, response_texts)
    
    # 整理topk结果
    sorted_items = sorted(
        [(i, f"视角{i+1}", weights[i].item(), complexities[i]) for i in range(p)],
        key=lambda x: x[2], reverse=True
    )
    top_indices = [item[0] for item in sorted_items[:topk]]
    top_perspectives = [
        {
            "视角": f"视角{idx+1}",
            "权重": weights[idx].item(),
            "复杂度": complexities[idx],
            "回答": response_texts[idx]
        }
        for idx in top_indices
    ]
    
    return {
        "总视角数": p,
        "topk结果": top_perspectives,
        "所有视角权重": [(item[1], item[2]) for item in sorted_items],
        "复杂度指标": [(item[1], item[3]) for item in sorted_items],
        "参数配置": {
            "complexity_weight_ratio": complexity_weight_ratio,
            "length_weight": length_weight,
            "diversity_weight": diversity_weight,
            "variance_weight": variance_weight,
            "temperature_settings": temperatures
        }
    }

if __name__ == "__main__":
    
    metaprompt = """

"""

    # 手动设置每次并发的温度值
    # manual_temperatures = [0.2, 0.4, 0.6, 0.8, 1, 1.4,1.8,2]
    # manual_temperatures = [0.2, 0.4, 0.6, 0.8, 1, 1.4, 2]
    manual_temperatures = [0.2, 0.5, 0.8, 1, 1.4, 2]
    # manual_temperatures = [0.2, 0.6, 0.8, 1.4, 2]
    # manual_temperatures = [0.2, 0.6, 1, 2]
    # manual_temperatures = [0.2, 0.8, 2] # 客服总结
    # manual_temperatures = [0.2, 2] # topk = 1

    
    # 执行多视角分析，将所有参数都放在函数调用中
    result = multi_perspective_analysis(
        metaprompt=metaprompt,
        p=len(manual_temperatures),  # 视角数量
        topk=2,  # 返回前2个视角
        temperature_settings=manual_temperatures,
        complexity_weight_ratio=0.6,  # 复杂度权重占比
        hidden_size=768,  # 隐藏层大小
        length_weight=0.4,  # 文本长度复杂度权重
        diversity_weight=0.5,  # 词汇多样性权重
        variance_weight=0.3  # 语义嵌入方差权重
    )

    # 输出结果（格式化JSON）
    print("\n=== 多视角分析最终结果 ===")
    print(json.dumps(result, ensure_ascii=False, indent=2))


In [ ]:
import json
from pydantic import KafkaDsn
import torch
import concurrent.futures
import re
from transformers import AutoModel, AutoTokenizer

class HierarchicalAggregationLayer(torch.nn.Module):
    def __init__(self, hidden_size=768, num_perspectives=3, attn_smoothing=0.1):
        """
        分层聚合层：通过三级处理融合多视角特征
        1) 两两视角组合的初级聚合
        2) 所有组合结果的全局聚合 
        3) 原始视角的注意力加权聚合
        
        参数：
            hidden_size (int): 特征向量的维度（默认768）
            num_perspectives (int): 需要聚合的视角数量（默认3）
            attn_smoothing (float): 注意力权重的标签平滑系数（默认0.1）
        """
        super().__init__()
        self.num_perspectives = num_perspectives
        self.attn_smoothing = attn_smoothing
        
        # 第一级聚合器：处理所有视角的两两组合
        # 共需要n*(n-1)/2个聚合器（n=视角数量）
        self.pairwise_aggregators = torch.nn.ModuleList([
            torch.nn.Sequential(
                torch.nn.Linear(hidden_size * 2, hidden_size),  # 将两个视角拼接后线性变换
                torch.nn.LayerNorm(hidden_size),               # 层归一化稳定训练
                torch.nn.GELU()                                # 高斯误差线性单元激活
            )
            for _ in range(num_perspectives * (num_perspectives - 1) // 2)
        ])
        
        # 第二级聚合器：整合所有两两组合的结果
        num_pairwise = num_perspectives * (num_perspectives - 1) // 2
        self.global_aggregator = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_pairwise, hidden_size * 2),  # 扩大维度增强表达能力
            torch.nn.LayerNorm(hidden_size * 2),                           # 归一化
            torch.nn.GELU(),                                               # 非线性激活
            torch.nn.Dropout(0.15),                                        # 随机失活防止过拟合
            torch.nn.Linear(hidden_size * 2, hidden_size)                  # 降维到原始维度
        )
        
        # 注意力机制：计算各原始视角的重要性权重
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_perspectives, hidden_size),  # 压缩视角拼接信息
            torch.nn.LayerNorm(hidden_size),                               # 归一化
            torch.nn.GELU(),                                              # 激活函数
            torch.nn.Linear(hidden_size, num_perspectives)                 # 输出各视角权重
        )
        
        self.softmax = torch.nn.Softmax(dim=-1)  # 将权重归一化为概率分布
        
    def forward(self, encoded_responses):
        """
        前向传播过程
        
        参数：
            encoded_responses: 多个视角的特征列表，每个元素形状为[hidden_size]
        
        返回：
            final_output: 聚合后的最终特征 [1, hidden_size]
            weights: 各视角的注意力权重 [1, num_perspectives] 
        """
        # 将多个视角特征堆叠为张量 [num_perspectives, hidden_size]
        stacked = torch.stack(encoded_responses)  
        
        # 第一阶段：两两视角聚合
        pairwise_outputs = []
        idx = 0  # 用于选择对应的聚合器
        
        # 遍历所有独特的视角组合对
        for i in range(self.num_perspectives):
            for j in range(i + 1, self.num_perspectives):
                # 拼接两个视角特征 [1, hidden_size*2]
                pair = torch.cat([stacked[i], stacked[j]], dim=-1).unsqueeze(0)
                
                # 通过对应的聚合器处理
                pairwise_outputs.append(self.pairwise_aggregators[idx](pair))
                idx += 1
        
        # 第二阶段：全局聚合
        # 拼接所有两两聚合结果 [1, hidden_size*num_pairs]
        pairwise_concat = torch.cat(pairwise_outputs, dim=-1)
        
        # 生成全局聚合特征 [1, hidden_size]
        global_repr = self.global_aggregator(pairwise_concat)
        
        # 第三阶段：注意力权重计算
        # 展平所有原始视角特征 [1, num_perspectives*hidden_size]
        concat = stacked.view(1, -1)
        
        # 计算归一化注意力权重 [1, num_perspectives]
        weights = self.softmax(self.attention(concat))
        
        # 应用标签平滑（正则化技术）
        if self.attn_smoothing > 0:
            uniform_dist = 1 / self.num_perspectives  # 均匀分布
            weights = weights * (1 - self.attn_smoothing) + self.attn_smoothing * uniform_dist
        
        # 计算原始视角的加权和 [hidden_size]
        weighted_aggregation = torch.sum(
            stacked * weights.t().unsqueeze(-1),  # 加权
            dim=0
        )
        
        # 最终输出：全局特征 + 加权原始特征 [1, hidden_size]
        final_output = global_repr + weighted_aggregation.unsqueeze(0)
        
        return final_output, weights

def load_encoder_model():
    """加载预训练的BERT编码器和分词器"""
    model_name = "D:\\xiangyue\\model\\bert-base-chinese"
    return AutoModel.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name)

def encode_text(text, model, tokenizer):
    """
    将文本编码为向量表示
    使用BERT模型提取[CLS]标记作为文本的整体表示
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():  # 不计算梯度以提高推理效率
        # 提取[CLS]标记的嵌入作为文本表示
        return model(**inputs).last_hidden_state[:, 0, :].squeeze(0)

def process_perspective(index, question, encoder_model, tokenizer, llm, temperature=0.1):
    """
    处理单个视角：
    1. 调用大语言模型生成特定视角的回答
    2. 将回答编码为向量表示
    返回包含索引、回答文本和编码向量的元组
    
    参数:
        index: 视角索引
        question: 问题文本
        encoder_model: 编码器模型
        tokenizer: 分词器
        llm: 大语言模型调用函数
        temperature: 控制生成随机性的温度值
    """
    # 使用特定视角前缀提示大语言模型，并传入temperature参数
    response = llm(f"【换一种视角思考{index+1}】{question}", temperature=temperature)
    return index, response, encode_text(response, encoder_model, tokenizer)

def multi_perspective_analysis(metaprompt, p=3, topk=1, llm=None, temperature_settings=None):
    """
    多视角分析主函数：
    1. 并行生成多个视角的回答
    2. 对回答进行编码
    3. 使用增强聚合层计算各视角权重
    4. 选择权重最高的视角作为最终结果
    
    参数:
        metaprompt: 输入的问题/提示
        p: 视角数量
        topk: 返回topk个最佳视角
        llm: 大语言模型调用函数
        temperature_settings: 温度值设置，可以是单个值或长度为p的列表
    """
    # 如果未提供llm，则使用全局导入的llm_qwen
    if llm is None:
        llm = llm_qwen
    
    # 处理temperature设置
    if temperature_settings is None:
        temperatures = [0.1] * p  # 默认所有请求使用0.1
    elif isinstance(temperature_settings, (int, float)):
        temperatures = [temperature_settings] * p  # 单个值应用到所有请求
    elif isinstance(temperature_settings, (list, tuple)) and len(temperature_settings) == p:
        temperatures = temperature_settings  # 使用提供的温度列表
    else:
        raise ValueError("temperature_settings should be a single value or a list of length p")
    
    # 初始化聚合模型和编码器
    aggregation_model = HierarchicalAggregationLayer(hidden_size=768, num_perspectives=p)
    encoder_model, tokenizer = load_encoder_model()
    
    # 使用线程池并行处理多个，每个请求使用不同的temperature
    with concurrent.futures.ThreadPoolExecutor(max_workers=p) as executor:
        futures = [
            executor.submit(
                process_perspective, 
                i, 
                metaprompt, 
                encoder_model, 
                tokenizer, 
                llm, 
                temperatures[i]
            ) for i in range(p)
        ]
        results = [future.result() for future in futures]
    
    # 按索引排序确保顺序正确
    results.sort(key=lambda x: x[0])
    encoded_responses = [enc for _, _, enc in results]  # 提取编码向量
    response_texts = [resp for _, resp, _ in results]  # 提取回答文本
    for idx, (text, temp) in enumerate(zip(response_texts, temperatures), start=1):
        print(f"第{idx}个(temperature={temp:.2f})：{text}")
        
    if encoded_responses:
        # 聚合多视角信息并获取权重
        aggregated_output, weights = aggregation_model(encoded_responses)
        
        # 按权重排序视角
        sorted_items = sorted([(i, f"response_{i+1}", weights[0, i].item()) for i in range(p)], 
                             key=lambda x: x[2], reverse=True)
        
        # 获取topk视角
        top_indices = torch.topk(weights[0], topk).indices
        top_perspectives = [(f"response_{idx+1}", weights[0, idx].item(), response_texts[idx]) 
                           for idx in top_indices]
        
        # 构建最终标题（当前已注释掉）
        final_title = "\n".join(re.sub(r'【[^】]+】', '', content).strip() for _, _, content in top_perspectives)
        
        return {
            # "final_title": final_title,
            "top_perspectives": top_perspectives,
            # "temperature_settings": temperatures,  # 返回使用的温度设置
            # "all_weights": [(f"response_{i+1}", weights[0, i].item()) for i in range(p)]
        }
    return None


if __name__ == "__main__":
    
    
    metaprompt = """

"""
    
    # 手动设置每次并发的温度值
    manual_temperatures = [0.2, 0.4, 0.6, 0.8, 1, 1.4,1.8,2]
    # manual_temperatures = [0.2, 0.4, 0.6, 0.8, 1, 1.4, 2]
    # manual_temperatures = [0.2, 0.5, 0.8, 1, 1.4, 2]
    # manual_temperatures = [0.2, 0.6, 0.8, 1.4, 2]
    # manual_temperatures = [0.2, 0.6, 1, 2]
    # manual_temperatures = [0.2, 0.8, 2] # 客服总结
    # manual_temperatures = [0.2, 2] # topk = 1

    result = multi_perspective_analysis(
        metaprompt, 
        p=len(manual_temperatures),  # 自动根据温度列表长度确定p值
        topk=2, 
        llm=llm_qwen, 
        temperature_settings=manual_temperatures  # 传入手动设置的温度列表
    )
    
    print(json.dumps(result, ensure_ascii=False, indent=2))  # 以JSON格式输出结果

In [ ]:
import torch
import concurrent.futures
import re
import json
from typing import List, Tuple, Dict, Optional
from openai import OpenAI

class EnhancedAggregationLayer(torch.nn.Module):
    """
    增强型聚合层，用于多视角响应的语义聚合
    融合ParScale技术的注意力平滑机制，提升多视角分析的稳定性
    """
    def __init__(self, hidden_size=768, num_perspectives=3, parscale_smoothing=0.1):
        """
        初始化聚合层
        
        Args:
            hidden_size: 编码器输出的隐藏层维度，默认768（BERT-base配置）
            num_perspectives: 多视角分析的视角数量，默认3个视角
            parscale_smoothing: ParScale注意力平滑系数，防止极端权重，默认0.1
        """
        super().__init__()
        self.num_perspectives = num_perspectives
        self.hidden_size = hidden_size
        self.parscale_smoothing = parscale_smoothing
        
        # 定义聚合网络结构：线性变换+层归一化+激活函数+Dropout+输出权重
        self.aggregate_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_perspectives, hidden_size),
            torch.nn.LayerNorm(hidden_size),
            torch.nn.SiLU(),  # 平滑激活函数，替代ReLU
            torch.nn.Dropout(0.1),  # 防止过拟合
            torch.nn.Linear(hidden_size, num_perspectives)
        )
        self.softmax = torch.nn.Softmax(dim=-1)  # 权重归一化
        
    def forward(self, encoded_responses):
        """
        前向传播：计算多视角响应的聚合权重和综合表示
        
        Args:
            encoded_responses: 各视角响应的编码向量列表，形状为[p, h]
        
        Returns:
            aggregated_output: 聚合后的综合向量，形状为[h]
            weights: 各视角的注意力权重，形状为[1, p]
        """
        stacked_responses = torch.stack(encoded_responses)  # 堆叠视角编码为[p, h]
        concat_responses = stacked_responses.view(1, -1)  # 拼接为[1, p*h]用于特征提取
        
        raw_weights = self.aggregate_layer(concat_responses)  # 计算原始权重
        weights = self.softmax(raw_weights)  # 权重归一化
        
        # 应用ParScale注意力平滑：防止某个视角权重过大
        if self.parscale_smoothing > 0:
            uniform_weight = 1.0 / self.num_perspectives
            weights = weights * (1 - self.parscale_smoothing) + self.parscale_smoothing * uniform_weight
        
        # 加权求和生成综合表示
        weighted_sum = torch.sum(stacked_responses * weights.t(), dim=0)  # [p, h] * [p, 1] 后求和
        return weighted_sum, weights


# 初始化OpenAI客户端，连接到本地Qwen模型服务
client = OpenAI(api_key="0", base_url="http://192.168.106.26:20000/v1")

def llm_qwen(prompt, model="Qwen3", temperature=0.7, top_p=0.8, max_tokens=1024):
    """
    调用Qwen模型API生成响应
    
    Args:
        prompt: 输入提示词
        model: 模型名称，默认Qwen3
        temperature: 生成温度，控制随机性，默认0.7
        top_p: 核采样参数，控制生成多样性，默认0.8
        max_tokens: 最大生成长度，默认1024
    
    Returns:
        模型生成的文本响应
    """
    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )
    return chat_response.choices[0].message.content


def load_encoder_model():
    """
    加载文本编码器模型（BERT-base）用于语义向量化
    
    Returns:
        model: BERT模型实例
        tokenizer: 对应的分词器
    """
    from transformers import AutoModel, AutoTokenizer
    model_name = "/app/sda1/xiangyue/model/bert-base-chinese"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return model, tokenizer


def encode_text(text, model, tokenizer):
    """
    将文本转换为语义向量表示（使用BERT的[CLS]标记）
    
    Args:
        text: 输入文本
        model: BERT模型实例
        tokenizer: 分词器实例
    
    Returns:
        文本的语义编码向量，形状为[hidden_size]
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():  # 推理时关闭梯度计算
        outputs = model(**inputs)
    # 使用BERT的第一个标记（[CLS]）作为整体语义表示
    return outputs.last_hidden_state[:, 0, :].squeeze(0)


def process_perspective(index, question, encoder_model, tokenizer, llm_qwen):
    """
    处理单个视角：生成响应并进行语义编码
    
    Args:
        index: 视角索引（从0开始）
        question: 分析问题
        encoder_model: 编码器模型
        tokenizer: 分词器
        llm_qwen: LLM调用函数
    
    Returns:
        index: 视角索引
        response: 模型生成的响应文本
        encoded_response: 响应的语义编码
    """
    # 为每个视角添加标识，引导模型从不同角度思考
    perspective_prompt = f"【视角{index+1}】{question}"
    response = llm_qwen(perspective_prompt)  # 调用Qwen模型生成响应
    encoded_response = encode_text(response, encoder_model, tokenizer)  # 编码响应文本
    return index, response, encoded_response


def multi_perspective_analysis(metaprompt, p=3, topk=1):
    """
    多视角分析主函数：生成多个视角响应并聚合分析结果
    
    Args:
        metaprompt: 元提示词（分析问题）
        p: 生成的视角数量，默认3个
        topk: 保留的关键视角数量，默认1个
    
    Returns:
        包含聚合结果、关键视角和所有视角权重的字典
    """
    # 初始化聚合模型与编码器
    aggregation_model = EnhancedAggregationLayer(hidden_size=768, num_perspectives=p)
    encoder_model, tokenizer = load_encoder_model()
    
    # 并行处理多个视角（使用线程池提高效率）
    with concurrent.futures.ThreadPoolExecutor(max_workers=p) as executor:
        futures = [
            executor.submit(
                process_perspective, i, metaprompt, encoder_model, tokenizer, llm_qwen
            )
            for i in range(p)
        ]
        # 收集所有视角的处理结果
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    # 按视角索引排序结果
    results.sort(key=lambda x: x[0])
    # 提取响应文本和编码向量
    responses = [(f"response_{i+1}", resp) for i, (_, resp, _) in enumerate(results)]
    encoded_responses = [enc for _, _, enc in results]
    response_texts = [resp for _, resp, _ in results]
    
    # 打印原始响应（调试和可视化）
    print("=== 原始响应 ===")
    for i, (key, response) in enumerate(responses):
        print(f"{key}: {response}")
    
    # 执行聚合逻辑（仅当有有效响应时）
    if encoded_responses:
        aggregated_output, weights = aggregation_model(encoded_responses)
        
        # 打印各视角的聚合权重
        print("\n=== 聚合权重 ===")
        for i, (key, _) in enumerate(responses):
            print(f"{key}: {weights[0, i].item():.4f}")
        
        # 获取权重最高的topk个视角
        top_weights, top_indices = torch.topk(weights[0], topk)
        print(f"\n=== Top {topk} 视角 ===")
        top_perspectives = []
        for i, idx in enumerate(top_indices):
            key = responses[idx][0]
            weight = top_weights[i].item()
            content = response_texts[idx]
            print(f"{key}: 权重={weight:.4f}\n内容: {content}")
            top_perspectives.append((key, weight, content))
        
        # 生成最终聚合结果（合并关键视角内容）
        final_title = "\n".join(re.sub(r'【[^】]+】', '', content).strip() for _, _, content in top_perspectives)
        print("\n=== 最终聚合结果 ===")
        print(final_title)
        
        return {
            "final_title": final_title,
            "top_perspectives": top_perspectives,
            "all_weights": [(responses[i][0], weights[0, i].item()) for i in range(p)]
        }
    else:
        print("没有有效的回答可供聚合")
        return None


if __name__ == "__main__":
    # 示例元提示词（空提示用于测试基本功能）
    metaprompt = """
/no_think
"""
    # 执行多视角分析（生成3个视角，保留1个关键视角）
    result = multi_perspective_analysis(metaprompt, p=3, topk=1)